In [162]:
import random
import math
import matplotlib.pyplot as plt
import atexit
from time import time, strftime, localtime
from datetime import timedelta

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    if elapsed:
        print(elapsed)

def endlog():
    end = time()
    elapsed = end-start
    return secondsToStr(elapsed)

start = time()
atexit.register(endlog)
log("")

list_fitness = []
# 7 kĩ năng
inputM = [10,0,10,10,0,10,0]
# cột mốc
milestone = [30,40,60,80,40,40,80]
achiM = []
for i,j in zip(milestone,inputM):
    achiM.append(i - j)
#[50, 30, 60, 90, 30, 100, 55]
Wf = [0.2,0.2,0.1,0.1,0.2,0.2]
Wp = [0,0,0,0,0,0.5,0.5]
S = [1,2,3,4,5,6,7]
B = 100000
course_rating = [2,5,5,5,2,4,4,3,3,5]
course_number_enroll = [68,46,47,32,81,20,66,82,62,24]
course_price = [99,95,47,76,9,13,94,57,5,17]
course = [1,2,3,4,5,6,7,8,9,10]
course_duration = [7,12,16,12,7,14,7,9,14,8]
course_min = [
    [0,0,10,0,10,0,0],
    [10,0,0,0,10,0,0],
    [0,10,0,0,0,0,10],
    [0,0,0,0,0,0,0],
    [10,0,0,10,0,0,0],
    [10,0,10,10,0,10,0],
    [0,0,0,0,0,0,0],
    [0,0,0,10,0,0,0],
    [10,0,0,10,10,0,0],
    [0,10,10,0,0,0,0]]
course_achi = [
    [0,10,26,0,43,0,8],
    [20,0,0,7,70,20,0],
    [10,60,0,0,14,0,84],
    [0,0,20,7,0,60,0],
    [40,0,0,40,10,0,11],
    [25,18,51,30,10,50,0],
    [10,0,10,0,0,0,74],
    [0,0,0,80,0,20,0],
    [60,20,0,20,30,0,20],
    [0,40,65,0,0,0,10]]

number_population = round(len(course) * 0.8)

def updateScoreNeeded(cur):
    t = []
    for i,j in zip(milestone,cur):
        t.append(i - j)
    return t

def updateCurrentScroce(cur,nex):
    t = []
    for i in range(0,len(cur)):
        t.append(cur[i] if cur[i] > nex[i] else nex[i])
    return t

def CanAttendCourse(cur,nex):
    t = []
    for i,j in zip(cur,nex):
        t.append(i - j)
    return sorted(t)[0] >= 0

class Choromosome(object):
    def __init__(self,genes,one_row):
        self.genes = genes
        self.one_row = one_row
        self.fitness = self.cal_fitness()
    
    def cal_fitness(self):
        f1 = 0; f2 = 0; f3 = 0; f4 = 0; f5 = 0;
        max_dura = 0; max_rating = 0; max_price = 0; max_enroll = 0; max_f2 = 0;
        column_skill = []
        g = []
        g_value = 0
        for i in self.one_row:
            max_dura = course_duration[i-1] if max_dura < course_duration[i-1] else max_dura
            max_rating = course_rating[i-1] if max_rating < course_rating[i-1] else max_rating
            max_price = course_price[i-1] if max_price < course_price[i-1] else max_price
            max_enroll = course_number_enroll[i-1] if max_enroll < course_number_enroll[i-1] else max_enroll
            max_f2 = len(course) * len(S)
            f1 += course_duration[i-1]
            for k in range(0,len(course_achi[i-1][::])):
                if(course_achi[i-1][k] != 0):
                    f2 += 1
            f3 += course_rating[i-1]
            f4 += course_number_enroll[i-1]
            f5 += course_price[i-1]
            column_skill.append(course_achi[i-1])
# chỗ này phải hỏi thêm
        for i in range(0,len(Wp)):
            if Wp[i] <= 0:
                continue
            val_column = [row[i] for row in column_skill]
            max_val_in_column = sorted(val_column)[-1]
            g.append(max_val_in_column)
            g_value += Wp[i] * max_val_in_column
#quy các objective value về 0...1
        
        max_dura *= len(self.one_row)
        max_rating *= len(self.one_row)
        max_price *= len(self.one_row)
        max_enroll *= len(self.one_row)
        number_deep_skill = len([i for i in Wp if i != 0])
        
        # phân vân 1/(f1/max_dura)
        f1 = 1/(max_dura/f1) * Wf[0]
        f5 = 1/(max_price/f5) * Wf[4]
        
        return f1 + Wf[1]*(f2/max_f2) + Wf[2]*(f3/max_rating) + Wf[3]*(f4/max_enroll) + f5 + Wf[5]*(number_deep_skill/len(Wp))
    
    def crossover(self,partner):
        index = random.randint(0,len(self.genes)-1)
        temp = self.genes[index]
        self.genes[index] = partner.genes[index]
        partner.genes[index] = temp
        prob = random.random()
        if prob < 0.45 and self.isValidChro():
            return self.fixChro()
        elif prob < 0.9 and partner.isValidChro():
            return partner.fixChro()
        else:
            return self.mutateChro() if prob < 0.5 else partner.mutateChro()
                
    def isValidChro(self):
        input_temp = inputM
        for i in self.genes:
            input_temp = updateCurrentScroce(input_temp,course_achi[i])
            if CanAttendCourse(input_temp,achiM):
                return True
        return False
    
    def fixChro(self):
#         for i in self.genes:
            
        return 0

    def mutateChro(self):
        return 0
    
def main():
    global list_fitness
    global number_population
    global achiM
    global inputM
    generation = 0
    isRun = True
    
    # initialize population
    population = []
    for i in range(number_population):
        genes = [[0 for i in range(len(course[::]))] for i in range(len(course[::])+1)]
        achiM_copy = achiM
        inputM_copy = inputM
        temp_curIn = inputM_copy
        course_choose_1_flow = []
        for j in range(0,len(achiM)):
            index = achiM.index(sorted(achiM)[-(j+1)])
            val = achiM_copy[index]
            skill_column = [row[index] for row in course_achi]
            temp = 0
            while temp < val:
                indexCourse = random.randint(1,len(skill_column)-1)
                achiSkillCourse = skill_column[indexCourse-1]
                if CanAttendCourse(temp_curIn,course_min[indexCourse-1]) and (indexCourse not in course_choose_1_flow) and achiSkillCourse > 0:
                    temp += achiSkillCourse
                    course_choose_1_flow.append(indexCourse)
                    temp_curIn = updateCurrentScroce(temp_curIn,course_achi[indexCourse-1])
                    achiM_copy = updateScoreNeeded(temp_curIn)
        # chuyển về ma trận biểu diễn tree
        # row đầu tiên
        temp_curIn = inputM_copy
        for i in course_choose_1_flow:
            if CanAttendCourse(temp_curIn,course_min[i-1]):
                genes[0][i-1] = 1
        for i in range(1,len(course_choose_1_flow)):
            temp_curIn = inputM_copy
            if not CanAttendCourse(temp_curIn,course_min[course_choose_1_flow[i]-1]):
                for j in range(0,i):
                    temp_curIn = updateCurrentScroce(temp_curIn,course_achi[course_choose_1_flow[j]-1])
                    if CanAttendCourse(temp_curIn,course_min[course_choose_1_flow[i]-1]):
                        genes[course_choose_1_flow[j]][course_choose_1_flow[i]-1] = 1
                        for k in range(0,i):
                            if 1 not in genes[course_choose_1_flow[i-k-1]] and genes[0][course_choose_1_flow[k]-1] != 1:
                                genes[course_choose_1_flow[i-k-1]][course_choose_1_flow[i]-1] = 1
                        break
                
        print(course_choose_1_flow)
        print(genes)
        population.append(Choromosome(genes,course_choose_1_flow))
    
    for i in range(0,len(population)):
        print(population[i].fitness)

#     while isRun:
#         population = sorted(population, key = lambda x:x.fitness)
#         new_generation = []
#         s = int((10*number_population)/100)
#         new_generation.extend(population[:s])
        
#         s = int((90*population_size)/100)
#         for _ in range(s):
#             parent1 = random.choice(population[:round(population_size*0.5)])
#             parent2 = random.choice(population[:round(population_size*0.5)])
#             child = parent1.crossover(parent2)
#             new_generation.append(child)
  
#         population = new_generation
                
#         print("Generation: {0}\tFitness: {1}\tPopulation: {2}".
#         format(generation,
#         population[0].fitness,len(population)))
        
        
#         list_fitness.append(population[0].fitness)
    
    
if __name__ == '__main__':
    main()
    
print("\n\nExcution time: {0}s".format(endlog()))

[5, 9, 7, 8, 1, 6, 3]
[[0, 0, 0, 0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[7, 5, 2, 4, 6, 1, 9, 3]
[[0, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[7, 5, 2, 9, 6, 4, 3]
[[0, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 

In [151]:
course = [
    [0,10,26,0,43,0,8],
    [20,0,0,7,70,20,0],
    [10,60,0,0,14,0,84],
    [0,0,20,7,0,60,0],
    [40,0,0,40,10,0,11],
    [25,18,51,30,10,50,0],
    [10,0,10,0,0,0,74],
    [0,0,0,80,0,20,0],
    [60,20,0,20,30,0,20],
    [0,40,65,0,0,0,10]]
print(len(course[::]))
# for i in range(0,len(course[0][::])):
#     print(i)
    
# listt = [60,40,70,100,40,120]
# for i in range(0,len(listt)):
#     print(sorted(listt)[-(i+1)])
# for i in range(0,len(course[0][::])):
#     column = [row[i] for row in course]
#     print(column)


10


In [129]:
Wp = [0,0,0,0,0,0.5,0.5]
print(len([ind for ind in Wp if ind != 0]))

2
